<a href="https://colab.research.google.com/github/sleiph/exploracao-de-dados/blob/main/precos_BOVESPA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preço das ações BOVESPA - yahoo finance

In [1]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime

## entrada de dados

In [2]:
# nome do fundo estudado.SA : nome a ser exibido no dataFrame
fundos = {
    'XPCI11.SA': 'XP Credito Imobiliario', 'XPLG11.SA': 'XP LOG', 'XPML11.SA': 'XP MALLS',
    'GRLV11.SA': 'CSHG GR Louveira (HF)', 'HGRU11.SA': 'CSHG Renda Urbana',
    'VILG11.SA': 'Vinci Logistica', 'VINO11.SA': 'Vinci Offices',
    'HCTR11.SA': 'Hectare Ce',
    'KNIP11.SA': 'Kinea Renda Imobiliária'
}

# data pro começo dos dados YYYY-mm-dd
data_inicio = '2007-06-01'

# método de ajuste
ajuste = 'Adj Close'

## populando o dataFrame

In [3]:
# pegando os dados da API
def getEstatisticas(bvmf, comeco, fim):
  url = 'https://query1.finance.yahoo.com/v7/finance/download/' + bvmf + '?period1=' + str(comeco) + '&period2=' + str(fim) + '&interval=1d&events=history'
  return pd.read_csv(url)

In [ ]:
# converte pra unix timestamp.
inicio = pd.to_datetime([data_inicio]).astype(int)[0]//10**9
# pega da data de recorte até o dia de hoje 
final = pd.to_datetime([datetime.today().strftime('%Y-%m-%d')]).astype(int)[0]//10**9

In [6]:
df = getEstatisticas('%5EBVSP', inicio, final)[['Date', ajuste]]

## adicionando o índice bovespa
df.rename(columns = {'Adj Close': 'IBovespa'}, inplace = True)
df['IBovespa'] = df['IBovespa']/1000

for k, v in fundos.items():
  try:
    df_temp = getEstatisticas(k, inicio, final)[['Date', ajuste]]
    df_temp.rename(columns = {ajuste: v}, inplace = True)
    df = pd.merge(df, df_temp, how='left', on='Date')
  except BaseException as err:
    print("erro no fundo {0}: {1}".format(v, err))

## Adicionando as datas como colunas
df.rename(columns = {'Date': 'Data'}, inplace = True)
df['Data'] = pd.to_datetime(df['Data'], format='%Y/%m/%d')

In [ ]:
# primeiros 5 dias dos dados
df.head()

In [ ]:
# últimos 5 dias dos dados
df[len(df.index)-5:].head(5)

## Gráficos

In [ ]:
# Cotação x tempo - FIIs
tickers = list(df.drop(['Data'], axis = 1).columns)
plt.figure(figsize=(16,6))

for i in tickers:
  plt.plot(df['Data'], df[i])
plt.legend(tickers)
plt.grid()
plt.title("Cotação x tempo", fontsize = 25)
plt.show()

In [ ]:
# médias móveis trimestrais e anuais da XP Credito Imobiliario
plt.figure(figsize=(16,6))
plt.plot(df['Data'], df['XP Credito Imobiliario'].rolling(window = 90).mean())
plt.plot(df['Data'], df['XP Credito Imobiliario'], alpha = 0.8)
plt.plot(df['Data'], df['XP Credito Imobiliario'].rolling(window = 365).mean())
plt.grid()
plt.title('Cotações diárias e médias móveis XP Credito Imobiliario', fontsize = 15)
plt.legend(['Média móvel trimestral','Cotação diária','Média móvel anual'])
plt.show()

In [ ]:
# heatmap das fiis
plt.figure(figsize=(16, 8))
sns.heatmap(df.corr(), annot = True)
plt.show()

In [ ]:
# retorno diário dos fundos
retornos = pd.DataFrame()
for i in tickers:
  retornos[i] = df[i].pct_change()
retornos['Data'] = df['Data']
retornos.describe()

In [ ]:
# pairplot, sei lá pra que serve isso
sns.pairplot(retornos)
plt.show()

In [ ]:
# histplot da XP Credito Imobiliario
sns.histplot(retornos['XP Credito Imobiliario'].dropna())

In [ ]:
# retorno acumulado dos fundos
retornos_soma = pd.DataFrame()

for ticker in tickers:
    retornos_soma[ticker] = (retornos[ticker]+1).cumprod()
retornos_soma['Data'] = retornos['Data']

plt.figure(figsize=(16,6))
plt.plot(retornos_soma['Data'], retornos_soma.drop(['Data'], axis = 1), alpha = 0.9)
plt.legend(tickers)
plt.title("Retorno x tempo", fontsize = 15)
plt.grid()
plt.show()